<a href="https://colab.research.google.com/github/beatriz-sait07/Computacao-grafica/blob/main/2dTO3d_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retornando uma matriz q talvez seja um arquivo .obj

In [1]:
# Clonar repositório PIFuHD e Lightweight-Human-Pose-Estimation
!git clone https://github.com/facebookresearch/pifuhd
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

# Baixar checkpoint para pose estimation
%cd /content/lightweight-human-pose-estimation.pytorch/
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

# Baixar modelos pré-treinados do PIFuHD
%cd /content/pifuhd/
!sh ./scripts/download_trained_model.sh

# Instalar versão específica do PyTorch (se necessário) e demais pacotes
!pip install torch==2.5.0+cu124 torchvision==0.20.0+cu124 torchaudio==2.5.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install flask flask-cors pyngrok

# Ajustar arquivo que usa np.bool (desatualizado)
!sed -i 's/np.bool/bool/g' /content/pifuhd/lib/sdf.py


Cloning into 'pifuhd'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 222 (delta 92), reused 82 (delta 82), pack-reused 96 (from 1)
Receiving objects: 100% (222/222), 399.39 KiB | 19.02 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 124 (delta 21), reused 19 (delta 18), pack-reused 90 (from 1)
Receiving objects: 100% (124/124), 230.29 KiB | 10.97 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/lightweight-human-pose-estimation.pytorch
--2025-02-15 20:02:39--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 23.203.77.88, 2600:1413:a000:118a::a87, 

In [2]:
import os
import cv2
import torch
import numpy as np

# Flask
from flask import Flask, request, send_file, jsonify
from flask_cors import CORS

# Pyngrok
from pyngrok import ngrok

# Pose Estimation imports
%cd /content/lightweight-human-pose-estimation.pytorch/
import demo
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.load_state import load_state
from modules.keypoints import extract_keypoints, group_keypoints
from modules.pose import Pose

# Remover fundo
!pip install rembg pillow onnxruntime
from rembg import remove
from PIL import Image

def load_pose_model(pose_model_path='checkpoint_iter_370000.pth'):
    net = PoseEstimationWithMobileNet()
    checkpoint = torch.load(pose_model_path, map_location='cpu')
    load_state(net, checkpoint)
    net = net.cuda().eval()
    return net

def get_rect(net, image_path, height_size=512):
    """
    Gera o arquivo _rect.txt necessário para o PIFuHD quando usamos --use_rect.
    """
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts

    rect_path = image_path.replace('.%s' % (image_path.split('.')[-1]), '_rect.txt')

    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

    all_keypoints_by_type = []
    total_keypoints_num = 0
    for kpt_idx in range(num_keypoints):
        total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

    pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
    for kpt_id in range(all_keypoints.shape[0]):
        all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
        all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale

    rects = []
    for n in range(len(pose_entries)):
        if len(pose_entries[n]) == 0:
            continue
        pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
        valid_keypoints = []
        for kpt_id in range(num_keypoints):
            if pose_entries[n][kpt_id] != -1.0:
                pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
        valid_keypoints = np.array(valid_keypoints)

        if len(valid_keypoints) == 0:
            continue

        if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
            pmin = valid_keypoints.min(0)
            pmax = valid_keypoints.max(0)
            center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.float32)
            radius = int(0.65 * max(pmax[0] - pmin[0], pmax[1] - pmin[1]))
        elif (pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and
              pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0):
            center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.float32)
            radius = int(1.45 * np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
            center[1] += int(0.05 * radius)
        else:
            center = np.array([img.shape[1] // 2, img.shape[0] // 2], dtype=np.float32)
            radius = max(img.shape[1] // 2, img.shape[0] // 2)

        x1 = int(center[0] - radius)
        y1 = int(center[1] - radius)
        rects.append([x1, y1, 2 * radius, 2 * radius])

    np.savetxt(rect_path, np.array(rects), fmt='%d')
    return rect_path

# Agora voltamos pro /content/pifuhd pois a pasta apps/simple_test.py está lá
%cd /content/pifuhd/

import subprocess

def run_pifuhd(net, image_path, resolution=256):
    """
    1) Gera o arquivo _rect.txt (com get_rect)
    2) Roda o apps.simple_test
    3) Retorna o caminho do .obj
    """
    # Gera .txt
    get_rect(net, image_path)

    folder = os.path.dirname(image_path)
    filename_no_ext = os.path.splitext(os.path.basename(image_path))[0]
    obj_path = f'/content/pifuhd/results/pifuhd_final/recon/result_{filename_no_ext}_{resolution}.obj'

    cmd = [
      "python", "-m", "apps.simple_test",
      "-r", str(resolution),
      "--use_rect",
      "-i", folder
    ]
    # Importante: rodar com cwd=/content/pifuhd pra encontrar o pacote apps
    subprocess.run(cmd, check=True, cwd="/content/pifuhd")

    return obj_path

def remove_background(input_path, output_path):
    """
    Remove o fundo da imagem antes de processá-la no PIFuHD.
    """
    try:
        img = Image.open(input_path)
        img_no_bg = remove(img)
        img_no_bg.save(output_path)
        return output_path
    except Exception as e:
        print(f"Erro ao remover fundo: {e}")
        return None



/content/lightweight-human-pose-estimation.pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00
/content/pifuhd


In [3]:
# 1) Carrega modelo de pose (uma única vez)
pose_net = load_pose_model('/content/lightweight-human-pose-estimation.pytorch/checkpoint_iter_370000.pth')

# 2) Define a aplicação Flask
app = Flask(__name__)
CORS(app)

@app.route('/pifuhd', methods=['POST'])
def pifuhd_endpoint():
    """
    Requisição:
      curl -X POST -F "image=@/caminho/imagem.png" <URL>/pifuhd -o resultado.obj
    """
    if 'image' not in request.files:
        return jsonify({"error": "Nenhuma imagem recebida. Utilize multipart form field name='image'."}), 400

    # Salva a imagem em /content/pifuhd/sample_images
    # filename = "upload.png"  # se preferir, pode gerar nomes únicos
    # image_path = f'/content/pifuhd/sample_images/{filename}'

    # image_file = request.files['image']
    # image_file.save(image_path)

     # Caminhos dos arquivos
    original_filename = "upload.png"
    processed_filename = "upload_no_bg.png"
    original_path = f'/content/pifuhd/sample_images/{original_filename}'
    processed_path = f'/content/pifuhd/sample_images/{processed_filename}'

    # Salva a imagem original
    image_file = request.files['image']
    image_file.save(original_path)

    # Remove o fundo da imagem
    processed_path = remove_background(original_path, processed_path)
    if processed_path is None:
        return jsonify({"error": "Falha ao remover fundo da imagem."}), 500

    # Executa o PIFuHD
    try:
        obj_path = run_pifuhd(pose_net, processed_path, resolution=256)
    except subprocess.CalledProcessError as e:
        return jsonify({"error": f"Falha ao rodar PIFuHD: {str(e)}"}), 500

    # Retorna o arquivo .obj
    return send_file(obj_path, as_attachment=True)




<ipython-input-2-35e50aee515d>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pose_model_path, map_location='cpu')


In [4]:
# 3) ngrok - configure seu token
NGROK_AUTH_TOKEN = "2sm6vyo1h04eE0HtlJZOpzk7ixT_7Vb4Z3sUE3E6ipB4kn1yB"  # pegue em https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Before connecting, check if a tunnel is already running
active_tunnels = ngrok.get_tunnels()
if len(active_tunnels) > 0:
    # Reuse the existing tunnel
    public_url = active_tunnels[0].public_url
    print("Reusing existing tunnel:", public_url)
else:
    # Start a new tunnel
    tunnel = ngrok.connect(5000) # Store the tunnel object
    public_url = tunnel.public_url # Get the public URL from the tunnel object
    print("URL pública:", public_url)

# ... (rest of your code) ...

# After your app has finished running, close the tunnel
ngrok.disconnect(public_url) # Disconnect using the public URL

URL pública: https://5fa4-34-142-194-196.ngrok-free.app


Pegue a ***URL publica*** abaixo para conexão com o front

In [ ]:
public_url = ngrok.connect(5000).public_url
print("URL pública:", public_url)

# 4) Iniciar flask
app.run(host="0.0.0.0", port=5000)

URL pública: https://e054-34-142-194-196.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:09:25] "POST / HTTP/1.1" 404 -
100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 139GB/s]
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:12:14] "POST /pifuhd HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:13:50] "POST /pifuhd HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:14:27] "POST /pifuhd HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:16:40] "POST /pifuhd HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 20:17:18] "POST /pifuhd HTTP/1.1" 200 -
